# <center>Genome : Algorithms to find innovative leaders in technology</center>
### <center> $\text{Jid}\acute{e}$ $\text{Anene}$  </center>


## Motivation
According to the economic innovation literature investors lack domain knowledge and therefore investors underreact to "innovative knowledge," in the form of patents, patent applications, and patent citations. I posit that investors underreact to "innovative knowledge" regardless of the medium, be it patents, clinical trials, or scientific/technical newspaper articles. So, there could be a bunch scenarios in which extremely positive technical/scientific information is released to the public by a firm, but the market price for the firm lags behind the actual value. Or there could be situations in which there is a divergence between the sentiment in technical/scientific media releases and the news media coverage. To see this, consider the case  for company XXX, this company operates in an industry in which survival relies on being an innovative leader (pharmaceuticals, biotech, gene therapy etc.), and now suppose that the non-scientific new media coverage of company XXX holdings is positive, but the technical/scientific public releases tells a different story, further suppose that the stock price for company XXX seems to be following the non-scientific news coverage instead of the scientific. If this is the case than there is opportunity. For companies that depend on innovation traditional fundamental analysis may not be as important as the analysis of scientific/technical innovative releases. 

In the economic literature this type of information content is called "innovative efficiency," and it is measured as a combination between patents/patent citations scaled by research and development expenditures. There are many papers that find strong evidence between innovative efficiency and future returns, but these papers do not use machine learning techniques, and they focus on causality rather than prediction. This project will focus on prediction. 

Furthermore, the experts who inform the sophisticated investor may be ill-informed since time and time again we have seen that experts underperform the machine: the wine taster can't tell the difference between Jersey wine and 1920s French Riesling, the portfolio manager underperforms the rules based consistent eft/mutual fund, the AI outperforming doctors in diagnosing cancer. What makes you think an expert can understand scientific jargon? Since technical writing is more structured than regular news media text this type of textual data may be easier to process, and so, it seems promising that a ML algorithms can come to some robust predictions. 

With this reasoning in hand, the primary purpose of this project will be to develop a trading strategy based on the information gleaned from scientific mediums. Specifically, I will focus my efforts on analyzing companies that pursue technologies in gene engineering companies. I will apply deep learning and NLP techniques on the text of the patent applications, cluster news articles into scientific/non-scientific, and glean insights from these two different clusters.

## Very Brief Background on Genome Engineering
Genome engineering, is a type of genetic engineering in which DNA is inserted, deleted, modified or replaced in the genome of a living organism. Unlike early genetic engineering techniques that randomly inserts genetic material into a host genome, genome editing targets the insertions to site specific locations. 

*There are 9-Types of Molecular Scissor (probably more)*
- Cas9/Crispr (RNA)
- TALE (Protein) 
- Group II intron (RNA)
- Meganuclease (Protein) 
- Recombinase (Protien) 
- TtAgo (DNA) 
- </sub>&lambda;<sub>-beta/exo MAGE (DNA)
- RecACage (DNA)
- ZnF (Protein) 
    

# Data Sources

First, I collected over 500 patent application by hand from [AppFT](http://appft.uspto.gov/netahtml/PTO/index.html) that mentioned the genome editing technique "*Crispr*", and were issued between Jan 2018 - June 2018. Then I put the CPC Subgroup ID codes located within each patent into the the text document `.\data\cpc_data.txt`. 
> CPC stands for *Cooperative Patent Classification*, and the CPC code for a patent classifies the technical features of a patent. Patent classifications make it feasible to search quickly for documents about earlier disclosures similar to or related to the invention for which a patent is applied for, and to track technological trends in patent applications.

A data dictionary for the variables available through PatentView is located at `.\dictionary\PatenView_DataDict.xlsx`

I then created the API wrapper `.\src\api_wrapper.py` to query patent data from [PatentView API](http://www.patentsview.org/api/doc.html). There are 3 classes within this file the class `PatentView` gets data from the [PatentView API](http://www.patentsview.org/api/doc.html), `ListToQuery` allows us to put the CPC Subgroup ID from `.\data\cpc_data.txt` into a valid PatentView query, and `Json_to_DataFrame` converts the Json data returned from [PatentView API](http://www.patentsview.org/api/doc.html) to a Pandas DataFrame. 

The only meaningful text data that you can pull from the [PatentView API](http://www.patentsview.org/api/doc.html) is `patent_abstract`, so [PatentView API](http://www.patentsview.org/api/doc.html) data is primarily used to get numeric features and filter patents based on specific criteria. PatentView.com has [data downloads](http://www.patentsview.org/download/) within these data downloads are data not available through the API. 
Three datasets of interest: 
- `.\data\brf_sum_text.tsv.zip`(13.512 GB) : Contains the text of the patent summary
- `.\data\claim.tsv.zip` (11.457 GB): Contains the text of the patent claims
- `detail_desc_text` (39.40 GB): I believe this

>The **summary** is supposed to be directed toward the invention, rather than the disclosure as a whole. The summary may point out the advantages of the invention or how it solves problems previously existent in the art (and preferably indicated in the Background of the Invention). In chemical cases the summary should point out in general terms the utility of the invention. If possible, the nature and gist of the invention or the inventive concept should be set forth. 

>The **claims** are the heart of a patent, in that they define the limits of exactly what the patent does, and does not, cover. That is, the patentee has the right to exclude others from making, using or selling, only those things which are described by the claims. 

Since the **claims** seem to be the meat and bones of patents the analysis will focus on parsing `~.\data\claim.tsv.zip`  until `detail_desc_text` is recieved. 


## Feature Selection & Data Selection

### Data Selection 
`\src\ExtractData.py` calls on modules from `\src\api_wrapper.py` to get the data from 

In [4]:
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import pairwise_distances
import os, sys
import pandas as pd
sys.path.append(os.path.join(os.getcwd(),"src"))
from ExtractData import ExtractData
from BagofWords import BagOfWords

#Extract the data
DATA = ExtractData()
DATA.get_data()


The default setting is to pull 10,000 filtering by patent application dates between 2007-01-01 and 2018-10-31, cpc subgroub ID codes from `.\data\cpc_data.txt`, along with the following filters: 
- **examiner_group** $\le$1799 and **examiner_group** $\ge$3715
- **cpc_section_id**="G" or **cpc_section_id**="A" or **cpc_section_id**="C" 
- **assignee_type** = 2 or **assignee_type** = 3

> Examiner groups: The examiner group number correspond to `art units`, which is another patent classification type. Basically the examiner groups $\le$1799 are chemisty and bioTech/drug patents, and examiner groups $\ge$3715 corresponds to prosthesis and surgerical patents. Please see [Uspto](https://www.uspto.gov/patents-application-process/classes-arranged-art-unit-art-units-1764-2691) for a a full description of what this filter covers.

>CPC section ID:  A = Human Necessitites,  G = Physics, C = Chemistry. A patent can have multiple CPC section listed so this filter is not very effective since a patent gets included if it has atleast one section ID that meets the criteria. 

> Assignee type:  2 - US Company or Corporation,  3 - Foreign Company or Corporation

Note that in this context 10,000  is an arbitrary number, and the fact that PatentView gave back exactly 10,000 patents leads to the natural conclusion that the space of patent that satisfy this query is larger than 10,000.  


With these filters in place one would expect that only biotech/chemical/drug companies would be picked up, but from the cross tab below we see that Mechanical companies are picked up as well. This is slightly surprising, but with additional context it is expected; when searching the patent applications by hand there were some surprising companies pursuing innovations in genome engineering, Microsoft had submitted patent applications that deal with storing data in DNA, Monsanto had a bunch of applications out for editing crops, and even defense contractor Raeython had submitted applications concerning the creation of genetic circuits. 

In [7]:

#Get Nber Cross Tab
pd.crosstab(DATA.pdData["nber_category_id"],DATA.pdData["nber_category_title"])


nber_category_title,Chemical,Cmp&Cmm,Drgs&Med,Elec,Mech,Others
nber_category_id,,,,,,
1,3329,0,0,0,0,0
2,0,205,0,0,0,0
3,0,0,2221,0,0,0
4,0,0,0,430,0,0
5,0,0,0,0,2575,0
6,0,0,0,0,0,1240


### Feature Selection

For feature selection I used a generlized Bag-Of-Words model called the n-Gram model. This model works by selecting unique n-length sequence of words, and hence if n=1 the n-Gram model is exactly the Bag-of-Words model you know and love. With this generlized model we can pick up common phrases and context instead of common words. 

I also implement a part-of-speech tagger which is a method from the python library `nlkt` that reads text and assigns parts of speech to each word, such as noun, verb, adjective, etc.. 

In [10]:
from nltk import pos_tag
text = "And now for something completely different".split()
pos_tag(text)

[('And', 'CC'),
 ('now', 'RB'),
 ('for', 'IN'),
 ('something', 'NN'),
 ('completely', 'RB'),
 ('different', 'JJ')]

Here we see that *and* is CC, a coordinating conjunction; *now* and *completely* are RB, or adverbs; for is IN, a *preposition*; *something* is NN, a noun; and *different* is JJ, an adjective (*Example taken from [nlkt documentation](https://www.nltk.org/book/ch05.html)*). With these tools we can hopefully encode the textual information into a sparse matrix.

The file `src/BagofWords.py` contains the Bag-of-Word algoithms. The steps for processing the text data are as follows:
1. Patents have alot of numbered items so delete all decimal numbers: 123456789
2. Removing all non-alpha numberic characters : '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
3. Apply the porter stemmer which reduces words to the root. "punctuation" -> "punctuat"
4. Reduce the case of every word to lower. 
5. Delete random white space. 
6. Tokenize the remaining doc. 

After this either extract n-gram sequences from the tokenized document or apply the pos_tag to the tokenized document. 


In [13]:
from BagofWords import BagOfWords

#Get the claims_df
claims_df=DATA.CLAIMS_DF
#Get the corpurs for the claims doc
claims_corpus = list(claims_df["text"].astype(str))

#Get Pos-tagger Bag-of-Words Varient for claims data. 
claims_bow = BagOfWords(pos_method = True, numbers = False)
claims_bow.get_bagofwords(claims_corpus)
claims_bow.set_varient_matrices()
#Tf-idf with sublinear tf scaling
pos_X_tfidf_subl = claims_bow.X_tfidf_subl

# Get 3-Gram data. 
claims_bow = BagOfWords(n = 3, numbers = False)
claims_bow.get_bagofwords(claims_corpus)
claims_bow.set_varient_matrices()
bag3_X_tfidf_subl = claims_bow.X_tfidf_subl

There are a bunch of ways we can normalize the count data outputted by the Bag-of-words model. 
For this illustration X_tfidf_subl (Tf-idf with sublinear tf) is the only one we will use since we only have finite amount of pages and time, and this can get boring. In `src/BagofWords.py` other methods are available. Please see [Stanfords NLP Link](https://nlp.stanford.edu/IR-book/html/htmledition/tf-idf-weighting-1.html) for a description of how Tf-idf weighting works, and [another Stanford Link](https://nlp.stanford.edu/IR-book/html/htmledition/sublinear-tf-scaling-1.html) to see what is up with Tf-idf Sublinear tf scaling. (Both links aren't very technical). 

### Dimension Reduction & Clustering Algorithms


The dataset outputed by `src/BagofWords.py` is very large and very sparse so we need to reduce the dimentions, therefore applications of dimension reductions techniques such as K-means and PCA is necessary, for K-means the [insert me] method is used to find the optimal cluster, $K^*$.
> K-means clustering works by defining clusters that minimuize the wintion-cluster quadratic cost function. $\min(\sum_{k=1}^k W(C_k))$ where $C_k$ is the $k_{th}$ cluster and $W(C_k)$ is the within-cluster quadratic cost. 

>**Elbow Method for for finding $K^*$**
- Compute clustering algorithm (e.g., k-means clustering) for different values of k.
- For each k, calculate the total within-cluster sum of square (wss)
- Plot the the of the within-cluster quadratic cost
- The location of a bend (knee) in the plot is generally considered as an indicator of the appropriate number of clusters.

Below see the elbow method graph and the  average silhouette method which I will explain below. 

In [ ]:
from ClusteringAlgos import ElbowMethod

#Get K-means: Elbow method 20 - 300, with Part-of-Speach Tagged TFid-Sublinear Text-Feature Matrix
tfid_kmeanModel = ElbowMethod(DATA,end=400,start = 20, steps = 20 ,note="pos_tfid_subl_20_300", graph_note= "Pos-Tagged Tfid Sublinear")
tfid_kmeanModel.run(pos_X_tfidf_subl)


#Get K-means: Elbow method 20 - 300, with 3-Gram TFid-Sublinear Text-Feature Matrix
bag3_kmeanModel = ElbowMethod(DATA,end=400,start = 20, steps = 20 , note="bag3_tfid_subl_20_300", graph_note= "3-Ngram Tagged Tfid Sublinear")
bag3_kmeanModel.run(bag3_X_tfidf_subl)



#Note: The above 2 algorithms take like 2 hours to run... would not reccomend running. 


 k-mean perc finish:  10% k-mean perc finish:  15% 

It seems that the elbow graph does not have the kink needed for identification of optimal $K^*$, and so there is no clear answer, and there the [average silhouette approach](http://www.sthda.com/english/wiki/print.php?id=239) is a more appropriate model for cluster selection.
> The average silhouette method computes the average silhouette of observations for different values of k. The optimal number of clusters $k$ is the one that maximize the average silhouette over a range of possible values for $k$ (Kaufman and Rousseeuw [1990]).

### Conclusion 

Ultimately  since dimension reduction in this instance is for later processing for a downstream phase of the model, cluster paramterization and cluster model selection highly depends on our predictions downstream. Meaning, once we merge our patent data set with stocks we then can evaluate each method and choose the method that gives the most robust predections. 

Unfortunately, its not straight forward exercise to match stock data with patent view data. Patents in PatentView come with the variables `assignee_organization` and `assignee_name` if a company is sponsoring a patent, then the company names appear in these variables. Frequently, patents assigned to agiven organization may show alternate spelling or syntax formats;
for example, some patents assigned to International Business Machines Corp. may simply show IBM. For a stupid machine this is very confusing. At [NBER](http://www.nber.org/patents/) there is a cross-walk datafile that allows a merge between PatentView data and a standardized schema for company identifiers. So fixing the issue of company naming conventions is an almost-trivial exercise, but matching company names to tickers is again a problem. Stock tickers are not unique over time, and the spelling of the company will again be misaligned for the same reasons specified before. So to finish the match one would need to do a fuzzy match, this type of match gives probabilities to two words refering to the same thing, and  we match all entries that are above a certain threshold. 

**There are a couple things in this project that need to be refined**:
- A more precise and systematic way of extracting the companies we want from PatentView.com. Instead of using the hand selected CPC Subgroup codes located in `.\data\cpc_data.txt`, apply a graph search algorthim. The nodes of the graph would be a patent, and the edges of the node could either be the cited patents or all patents that are in the same CPC subgroup technology class.  
- Refining the bag-of-words models.
- Incorporating code to allow nonnegative matrix factorization for dimension reduction, in Jeremy Watt's book it is mentioned that these techniques are commonly applied to NLP. 
- General UI changes so that the software is easier to work with. 

**The projects current direction**
1. Use features from patentview.com and dimensionally reduced text-matrix to predict stock prices, and then form a medium-to-long trading strategy based off of predictions. 
2. Use the TwoSigma data set on kaggle.com that has stock prices + news paper articles and analyze divergence between semtiment in the news vs sentiment in patents. Form a medium-to-short term strategy off of this information,
3. Generlize to [Clinical Trial](https://aact.ctti-clinicaltrials.org/download) data. Repeat step 1 and 2 with clinical trial data.
    - Less finance: Predict if clinical trial will recieve FDA Approval. 
4. Move to options. 

### Final Words
I picked this project primarily because I heard an NPR podcast about the genome editing technique *Crispr* a couple years back and I was immediately hooked. Genome editing is going to be shocklingly transformative, and already technology, defense, bioTech, engineering, agricultural companies are using these techniques. 

Here a cool story before eletricity there was steam. Companies back in the early 1900s used huge steam engines to power everything. The mechanical power came from a single massive steam engine, which turned a central steel drive shaft that ran along the length of the factory. The whole factory was connected to this big engine, and so steam engines rarely stopped. If a single machine in the factory needed to run, the coal fires needed to be fed. Very stupid, why not use an iphone? Electricity allowed power to be delivered exactly where and when it was needed, and early adopters of eletricy to their specific business needs easily dispatched of the their steam-engine foes. The companies Ford and Oldsmobile were the early adopters of the eltricity wave and its applications to the assembly line. Ransom Olds, patented the assembly line concept, which he put to work in his Olds Motor Vehicle Company factory in 1900s. The moving assembly line was developed for the Ford Model T and began operation on in 1913. The success of assembly line dominated and spread worldwide, and by 1930 a company that did not adopted would risk being broke by not being able to compete. 

 The moral of this story is that investors can underreact to innovative knowledge, but companies can can also be slow to adopt and the leaders in adopting transformative technology often dominate. This project main purpose is to find early adopters and leaders in innovation, and so this project can be condensly formalized as 'algorithms to find innovative leaders in classes of technology.' I've had suggestions to make this tool more general and apply it to other technologies such as Bitcoin. I may do this, but I think the impact of genome engineering will far outpace a novel technique to make a marketplace decentralized. So I will focus on genome engineering, and then generlize after I finish with my initial goals. I may set up a website, who knows? 







